# 05. PyTorch Going Modular Exercises

Welcome to the 05. PyTorch Going Modular exercise template notebook.

There are several questions in this notebook and it's your goal to answer them by writing Python and PyTorch code.

> **Note:** There may be more than one solution to each of the exercises, don't worry too much about the *exact* right answer. Try to write some code that works first and then improve it if you can.

## Resources and solutions

* These exercises/solutions are based on [section 05. PyTorch Going Modular](https://www.learnpytorch.io/05_pytorch_going_modular/) of the Learn PyTorch for Deep Learning course by Zero to Mastery.

**Solutions:**

Try to complete the code below *before* looking at these.

* See a live [walkthrough of the solutions (errors and all) on YouTube](https://youtu.be/ijgFhMK3pp4).
* See an example [solutions notebook for these exercises on GitHub](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/solutions/05_pytorch_going_modular_exercise_solutions.ipynb).

## 1. Turn the code to get the data (from section 1. Get Data) into a Python script, such as `get_data.py`.

* When you run the script using `python get_data.py` it should check if the data already exists and skip downloading if it does.
* If the data download is successful, you should be able to access the `pizza_steak_sushi` images from the `data` directory.

In [4]:
%%writefile going_modular/get_data.py
# YOUR CODE HERE
import os
import urllib.request
import zipfile

project_folder = 'going_modular'
data_folder = os.path.join(project_folder, 'data')

if not os.path.isdir(project_folder):
  os.mkdir(project_folder)

if not os.path.isdir(data_folder):
  os.mkdir(data_folder)

zip_url = "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip"
zip_name = os.path.basename(os.path.normpath(os.path.join(data_folder, zip_url)))

if not os.path.isfile(zip_name):
  urllib.request.urlretrieve(zip_url, "pizza_steak_sushi.zip")
  print(f"Descargando archivo {zip_name}")
else:
  print(f"{zip_name} ya se encuentra descargado")

with zipfile.ZipFile(zip_name, 'r') as zip_ref:
    zip_ref.extractall(data_folder)

os.remove(zip_name)
print(f"{zip_name} fue descomprimido en {data_folder}")

Writing going_modular/get_data.py


In [5]:
# Example running of get_data.py
!python going_modular/get_data.py

Descargando archivo pizza_steak_sushi.zip
pizza_steak_sushi.zip fue descomprimido en going_modular/data


## 2. Use [Python's `argparse` module](https://docs.python.org/3/library/argparse.html) to be able to send the `train.py` custom hyperparameter values for training procedures.
* Add an argument flag for using a different:
  * Training/testing directory
  * Learning rate
  * Batch size
  * Number of epochs to train for
  * Number of hidden units in the TinyVGG model
    * Keep the default values for each of the above arguments as what they already are (as in notebook 05).
* For example, you should be able to run something similar to the following line to train a TinyVGG model with a learning rate of 0.003 and a batch size of 64 for 20 epochs: `python train.py --learning_rate 0.003 batch_size 64 num_epochs 20`.
* **Note:** Since `train.py` leverages the other scripts we created in section 05, such as, `model_builder.py`, `utils.py` and `engine.py`, you'll have to make sure they're available to use too. You can find these in the [`going_modular` folder on the course GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/going_modular/going_modular).

In [6]:
import shutil

scripts_to_download = ["https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/going_modular/going_modular/model_builder.py",
                       "https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/going_modular/going_modular/utils.py",
                       "https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/going_modular/going_modular/engine.py",
                       "https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/going_modular/going_modular/data_setup.py"]

for i in scripts_to_download:
  project_folder = 'going_modular'
  file_name = os.path.basename(os.path.normpath(i))

  if not os.path.isfile(os.path.join(project_folder, file_name)):
    urllib.request.urlretrieve(i, file_name)
    shutil.move(file_name, os.path.join(project_folder, file_name))
    print(f"Descargando archivo: {file_name} en {project_folder}")
  else:
    print(f"{file_name} ya se encuentra descargado")

Descargando archivo: model_builder.py en going_modular
Descargando archivo: utils.py en going_modular
Descargando archivo: engine.py en going_modular
Descargando archivo: data_setup.py en going_modular


In [7]:
%%writefile going_modular/train.py
# YOUR CODE HERE
import engine, model_builder, data_setup
import argparse
import torch
from torchvision.transforms import v2
import torch.nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'

parser = argparse.ArgumentParser()
parser.add_argument("--num_epochs", help="Number of epochs")
parser.add_argument("--batch_size", help="Batch size")
parser.add_argument("--hidden_units", help="Number of hidden units")
parser.add_argument("--learning_rate", help="Learning rate for Adam")

args = parser.parse_args()

transforms = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.uint8, scale=True),
    v2.RandomResizedCrop(size=(64, 64), antialias=True),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir='going_modular/data/train',
                                                                  test_dir='going_modular/data/test',
                                                                  transform=transforms,
                                                                  batch_size=int(args.batch_size))

model = model_builder.TinyVGG(3, int(args.hidden_units), len(class_names)).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=float(args.learning_rate))
loss_fn = torch.nn.CrossEntropyLoss()

results = engine.train(model=model,
                       train_dataloader=train_dataloader,
                       test_dataloader=test_dataloader,
                       optimizer=optimizer,
                       loss_fn=loss_fn,
                       epochs=int(args.num_epochs),
                       device=device)

Writing going_modular/train.py


In [8]:
# Example running of train.py
!python going_modular/train.py --num_epochs 5 --batch_size 128 --hidden_units 128 --learning_rate 0.0003

  0% 0/5 [00:00<?, ?it/s]Epoch: 1 | train_loss: 1.1018 | train_acc: 0.2899 | test_loss: 1.0917 | test_acc: 0.3733
 20% 1/5 [00:02<00:11,  2.99s/it]Epoch: 2 | train_loss: 1.0407 | train_acc: 0.4546 | test_loss: 1.0721 | test_acc: 0.4000
 40% 2/5 [00:04<00:05,  1.87s/it]Epoch: 3 | train_loss: 0.9566 | train_acc: 0.5374 | test_loss: 1.1139 | test_acc: 0.4133
 60% 3/5 [00:05<00:02,  1.48s/it]Epoch: 4 | train_loss: 0.9704 | train_acc: 0.4921 | test_loss: 1.0784 | test_acc: 0.4533
 80% 4/5 [00:06<00:01,  1.29s/it]Epoch: 5 | train_loss: 1.0013 | train_acc: 0.5763 | test_loss: 0.9501 | test_acc: 0.4667
100% 5/5 [00:07<00:00,  1.42s/it]


## 3. Create a Python script to predict (such as `predict.py`) on a target image given a file path with a saved model.

* For example, you should be able to run the command `python predict.py some_image.jpeg` and have a trained PyTorch model predict on the image and return its prediction.
* To see example prediction code, check out the [predicting on a custom image section in notebook 04](https://www.learnpytorch.io/04_pytorch_custom_datasets/#113-putting-custom-image-prediction-together-building-a-function).
* You may also have to write code to load in a trained model.

In [9]:
# YOUR CODE HERE
import torch
from torchvision.transforms import v2
from PIL import Image
import torch.nn as nn

parser = argparse.ArgumentParser()
parser.add_argument("--image", help="Image to predict label")

args = parser.parse_args()

pil_img = Image.open(args.image)

transforms = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.uint8, scale=True),  # optional, most input are already uint8 at this point
    v2.RandomResizedCrop(size=(64, 64), antialias=True),  # Or Resize(antialias=True)
    v2.ToDtype(torch.float32, scale=True),  # Normalize expects float input
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

img_transformed = transforms(pil_img)

softmax = nn.Softmax(dim=1)

with torch.inference_mode():
  y_logits = model(img_transformed)
  y_preds = torch.argmax(softmax(y_logits), dim=1)

prediction = class_to_idx[y_preds.item()]

In [10]:
# Example running of predict.py
!python predict.py --image data/pizza_steak_sushi/test/sushi/175783.jpg

python3: can't open file '/content/predict.py': [Errno 2] No such file or directory
